<a href="https://colab.research.google.com/github/linusmtz/hackmty2023/blob/main/Clasificaci%C3%B3n_de_clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de Clientes basados en sus datos
El siguiente notebook describe el proceso llevado acabo por el equipo Binary Reapers (Mesa 2-D) durante el HackMTY para la creación de un sistema de Inteligencia Artificial por medio de Redes Neuronales Supervisadas para la clasificación de Clientes en las diferentes categorías de importancia de un banco, con el fín de poder asesorarlos financieramente de manera adecuada dependiendo de su situación actual.

Para esto utilizaremos como base el conjunto de datos sobre clientes de datos proveniente del Repositorio de Machine Learning de la UCI: https://archive.ics.uci.edu/dataset/222/bank+marketing

Primero importaremos la librería de Datasets de la UCI dentro de nuestro modelo.

In [1]:
pip install ucimlrepo

Primeramente, utilizaremos como Dataset la información de 45,211 clientes de bancos, esta información incluye las siguientes campos de datos:

